In [33]:
import pytz
import pandas as pd
from datetime import datetime
from timezonefinder import TimezoneFinder

def find_UTC_timezone_for_coordinates(longitude, latitude):
    
    # Get the time zone based on the coordinates
    timezone_str = tf.timezone_at(lng=longitude, lat=latitude)

    # Convert the timezone string to a timezone object using pytz
    timezone = pytz.timezone(timezone_str)

    # Convert the timezone string to a timezone object using pytz
    time_in_timezone = datetime.now(timezone)
    
    # Get the UTC offset in seconds
    utc_offset_seconds = time_in_timezone.utcoffset().total_seconds()
    
    # Convert the offset to hours
    utc_offset_hours = utc_offset_seconds / 3600
    
    return [timezone, utc_offset_hours]


def turn_offset_into_string(value):
    zfilling = str(int(value)).zfill(2)
    if value>=0:
        return "+{0}:00".format(zfilling)
    else:
        return "-{0}:00".format(zfilling)
    

################
# Main program #
################

# Requires: pip install timezonefinder pytz

# First we find the UTC time zone and offset associated 
path_in_csv = r"/home/jovyan/work/private/sources/projects/wow4mod/reference_tables/WOW_EUMETNET_CWS_Locations_2020_Within_Elevation.csv"
path_ou_csv = path_in_csv
df = pd.read_csv(path_in_csv, sep=";")

tf = TimezoneFinder()
df[['UTC_TimeZone', 'UTC_OffsetHours']] = [item for item in df.apply(lambda x: find_UTC_timezone_for_coordinates(x["longitude"], x["latitude"]), axis=1)]
df['UTC_OffsetHours'] = df.apply(lambda x: turn_offset_into_string(x["UTC_OffsetHours"]), axis=1)

# Now, we find a name for the stations that is integer-based
df["HARP_SID_int"] = 41*10**8 + df["rowid"] + 10**8
print(df.head())

df.to_csv(path_ou_csv, sep=";")

   rowid                               Site Id  longitude  latitude  IsWithin  \
0      1                                   NaN    -2.9194   54.1866      True   
1      2  0029ead6-4cc1-ea11-bf21-0003ff5982ee     3.6398   51.1036      True   
2      3  0035f568-ee98-e911-80e7-0003ff59889d     4.7110   50.8769      True   
3      7  00a30701-f954-e911-b048-0003ff59ad54    -0.1275   51.5072      True   
4      9  00c55df9-5069-ea11-99e5-0003ff59b16f    16.7794   60.2276      True   

  CountryId                 geometry  elevation      UTC_TimeZone  \
0        UK  POINT (-2.9194 54.1866)       37.0     Europe/London   
1        BE   POINT (3.6398 51.1036)        5.0   Europe/Brussels   
2        BE    POINT (4.711 50.8769)       38.0   Europe/Brussels   
3        UK  POINT (-0.1275 51.5072)       14.0     Europe/London   
4        SE  POINT (16.7794 60.2276)       39.0  Europe/Stockholm   

  UTC_OffsetHours  HARP_SID_int  
0          +00:00    4200000001  
1          +01:00    420000000